## Problem

A little while later you stumble upon a population of wild rabbits of different colour that fascinates Dr. Michael, the population geneticist. He is fascinated by the colours ranging from blue to red. He has a task in mind but as with Dr. Dwight, he doubts whether you can solve the task, so he wants to test you on your knowledge of Mendel’s Laws and probability. He asks the following question:

Suppose we have a single ancestor with the genotype AaBb in the 0th generation. The ancestor mates with another individual with genotype AaBb to produce offsprings. In the first generation the ancestor has 2 children. At each generation, a successor organism mates with a foreign individual with genotype AaBb to produce 2 children.

Given two positive integers k and N, return the probability that at least N AaBb organisms will belong to the k-th generation of the family tree (don’t count the AaBb foreign mates at each level).

Assume Mendel’s second law (independent assortment) holds for the factors. (You can try out simulating the family tree for a challenge or apply biology knowledge from your core years and basic probability to come up with a simpler solution!)

### Sample Input
- k = 2 
- N = 1

### Sample Output
0.684

## Solution

The simulation will obviously crash your computer for a fairly reasonable number. Thus, we need to find some way to avoid the simulation method. We will instead try to find a pattern and figure out how to model the question mathematically.

Let's first understand modeling the mating process.  

Let us assume we have two heterozygous organisms with genotypes `Aa Bb`. Independent assortment implies that the probability of an `aa BB` offspring is equal to $Pr(aa) × Pr(BB) = \frac{1}{4} × \frac{1}{4} = \frac{1}{16}$. The value $\frac{1}{4}$ comes from the fact that the probability of any offspring of this subtype (`AA`, `Aa`, or `aa` - same for `BB`, `Bb`, and `bb`) is equally likely from the crossing. 

Here is a cross to demonstrate the probabilities for each genotype after a single mating event (Parents: `AaBb` X `AaBb`):

|	|AB|	Ab|	aB|	ab|
|:--:|:--:|:--:|:--:|:--:|
|<b>AB</b>|	AA BB|	AA Bb|	Aa Bb|	Aa Bb|
|<b>Ab</b>|	AA bB|	AA bb|	Aa bB|	Aa bb|
|<b>aB</b>|	aA BB|	aA Bb|	aa BB|	aa Bb|
|<b>ab</b>|	aA bB|	aA bb|	aa bB|	aa bb|

**Note**: 
- Whatever maybe the genotype of the parent, they are bound to produce gametes belonging to one (or more) of the four types: `AB`, `Ab`, `aB`, `ab`
- Also, each row and column in the table above has exactly 1 `AaBb` individual. Thus, the probability of an `AaBb` individual in a given row is $\frac{1}{4}$

At each generation we mate the individuals with a foreign individual having the genotype `AaBb`. Thus, one of the axis of the matrix (row or column in the table above) is always fixed - `AB Ab aB ab`.  

From this and the *note* above we can deduce that irrespective of the genotype of the individual mating with the foreign individual - the probability of getting the genotype `AaBb` from any mating event is $\frac{1}{4}$

To solve the question presented, we have to compute the probability to observe at least $N$ `Aa Bb` offsprings after $k$ generations.   
We first need to compute the probability to observe $N$ `Aa Bb` offsprings after $k$ generations.  
As the probability of getting `AaBb` individuals is fixed, we can treat each mating event as a *Bernoulli trial*, where, the probability of success (getting `AaBb` individual) is $\frac{1}{4}$. The number of such trials for $k$-th generation will be $2^{k}$ (the total number of mating events in generation $k$). Hence, Probability of getting $N$ `AaBb` individuals in generation $k$:
$$Pr(N,k) = (^{2^{k}}_N)(\frac{1}{4})^{N}( 1−\frac{1}{4} ) ^{2^{k}−N}$$
which we recognize as the probability mass function of a random variable that follows a Binomial distribution $B(n,p)$, with $n = 2^{k}$ and $p = \frac{1}{4}$.   
Now, we need to find the probability that at least N such organisms will belong to the k-th generation, which is given by:  
$$1 − \sum_{x=0}^{N-1}Pr(X=x)$$  
where, $X$ is a random variable denoting the number of offspring, that is one minus all the cases $Pr(X≠N)$.

So we need to write functions combining both the equations above.  
One function will be to calculate the probability of $N$ individuals in generation $k$. The other function will call the first function and return the cumulative probability as mentioned above.

In [1]:
from math import factorial as fac

In [2]:
# Function to calculate binomial coefficient
def binomial_coefficient(n, r):
    try:
        return fac(n) // fac(r) // fac(n - r)
    # When r > n
    except ValueError:
        return 0

In [3]:
# Function to calculate probability of n individuals in k-th generation
def bernoulli_trials(n, k):
    return binomial_coefficient(2**k, n) * 0.25**n * 0.75**(2**k - n)

In [4]:
# Function to calculate the final probability
def final_probability(k, N):
    return 1 - sum(bernoulli_trials(n, k) for n in range(N))

In [5]:
final_probability(2, 1)

0.68359375